# "Deep" Neural Network with XOR problem

## using TensorBoard

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
xy = np.loadtxt('train_nn_xor.txt', unpack=False)
x_data = xy[:, 0:-1]
y_data = np.reshape(xy[:, -1], (4,1))
print(x_data)
print(y_data)

[[ 0.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  1.]]
[[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]]


In [3]:
X = tf.placeholder(tf.float32, name="X-input")
Y = tf.placeholder(tf.float32, name="Y-input")

In [4]:
# Deep network configuration.: Use more layers. 
W1 = tf.Variable(tf.random_uniform( [2,5], -1.0, 1.0),name = 'weight1')
W2 = tf.Variable(tf.random_uniform( [5,4], -1.0, 1.0),name = 'weight2')
W3 = tf.Variable(tf.random_uniform( [4,1], -1.0, 1.0),name = 'weight3')

In [5]:
b1 = tf.Variable(tf.zeros([5]), name="bias1")
b2 = tf.Variable(tf.zeros([4]), name="bias2")
b3 = tf.Variable(tf.zeros([1]), name="bias3")

In [6]:
# Define Histogram Summary
w1_histo = tf.histogram_summary("weight1", W1)
w2_histo = tf.histogram_summary("weight2", W2)
w3_histo = tf.histogram_summary("weight3", W3)

In [7]:
# Define Scalar Summary
b1_histo = tf.histogram_summary("biases1", b1)
b2_histo = tf.histogram_summary("biases2", b2)
b3_histo = tf.histogram_summary("biases3", b3)

In [8]:
y_histo = tf.histogram_summary("place_y", Y)

In [9]:
# Define Scope with Hypothesis
with tf.name_scope("layer2") as scope:
    L2 = tf.sigmoid(tf.matmul(X, W1) + b1)

with tf.name_scope("layer3") as scope:
    L3 = tf.sigmoid(tf.matmul(L2, W2) + b2)

with tf.name_scope("layer4") as scope:
    hypo = tf.sigmoid(tf.matmul(L3, W3) + b3)

In [10]:
# Define Cost Function with scope
with tf.name_scope("cost_function") as scope:
    cost = -tf.reduce_mean(Y*tf.log(hypo) + (1-Y)*tf.log(1. - hypo))
    cost_scar = tf.scalar_summary("cost", cost)


In [11]:
# Define Train Logic with scope
alpha = tf.Variable(0.1)
with tf.name_scope("train") as scope:
    optimizer = tf.train.GradientDescentOptimizer(alpha)
    train = optimizer.minimize(cost)


In [14]:
init = tf.initialize_all_variables()
with tf.Session() as sess:
    # Marge Summaries
    merged = tf.merge_all_summaries()
    writer = tf.train.SummaryWriter('./tensorboard/DeepNN', sess.graph)
    
    # init variables
    sess.run(init)
    
    for step in range(10001):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        if step % 2000 == 0:
            print("step : ", step)
            print("cost : ", sess.run(cost, feed_dict={X:x_data, Y:y_data}))
            print("W1 : \n", sess.run(W1))
            print("W2 : \n", sess.run(W2))
            print("W3 : \n", sess.run(W3))
            print("="*60)
            
            summary = sess.run(merged, feed_dict={X:x_data, Y:y_data})
            writer.add_summary(summary, step)
        
    
    # Test Model
    correct_prediction = tf.equal(tf.floor(hypo + 0.5), Y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    
    # Check Accuracy
    print("hypo : \n", sess.run(hypo, feed_dict={X:x_data, Y:y_data}))
    print("floor : \n", sess.run(tf.floor(hypo+0.5), feed_dict={X:x_data, Y:y_data}))
    print("correct_prediction : \n", sess.run(correct_prediction, feed_dict={X:x_data, Y:y_data}))
    print("accuracy : \n", sess.run(accuracy, feed_dict={X:x_data, Y:y_data}))
    

step :  0
cost :  0.697454
W1 : 
 [[-0.32017502  0.9640075  -0.22153488  0.42467871 -0.78454638]
 [ 0.12175141  0.29855728  0.13210329  0.69192827 -0.84961689]]
W2 : 
 [[ 0.52003384  0.19968994  0.67923504 -0.00841916]
 [-0.22266243 -0.74752688  0.397753    0.61091894]
 [-0.00541555 -0.35958916  0.99465752 -0.08501495]
 [-0.29192007 -0.44530201  0.84550834  0.69162631]
 [ 0.94730109  0.65942025 -0.36853722 -0.0107594 ]]
W3 : 
 [[ 0.68633747]
 [ 0.38644731]
 [ 0.32227474]
 [-0.98085344]]
step :  2000
cost :  0.692965
W1 : 
 [[-0.27486765  0.88891554 -0.1583562   0.29977921 -0.65714955]
 [ 0.14361     0.06179487  0.1580472   0.57834083 -0.73394376]]
W2 : 
 [[ 0.55401647  0.19787358  0.64442545  0.00829354]
 [-0.15028018 -0.71091747  0.45434186  0.5002234 ]
 [ 0.02850897 -0.35914567  0.96700114 -0.07507259]
 [-0.22212718 -0.4083465   0.88319558  0.57014483]
 [ 0.87291861  0.59125638 -0.50443006  0.27797732]]
W3 : 
 [[ 0.42654756]
 [ 0.03713532]
 [ 0.50209278]
 [-0.92343885]]
step :  4000


## Launch tensorboard 
* tensorboard --logdir=your tensorboard file